In [ ]:
import googlesearch
from newspaper import Article, news_pool
from datetime import timedelta, date, datetime
import arrow
from newspaper import Config
import io
import os
import time
import re
from langdetect import detect, detect_langs
from date_guesser import guess_date, Accuracy
import random

In [ ]:
def Get_google_urls(query, quantity, startdate, enddate):
    url_list = []
    pre_rejected_list = []
    current_date = startdate.datetime
    while current_date <= enddate:
        my_tbs = googlesearch.get_tbs(current_date, current_date)
        # for url in googlesearch.search_news(query, tld="com", tpe='news', stop=None, pause=2, lang='en', tbs=my_tbs):
        pause = random.randrange(15, 60)
        counter = 0
        for url in googlesearch.search(query, tld="com", tpe='nws', num=50, stop=300, pause=pause, lang='en',
                                       tbs=my_tbs):
            text = url.rstrip('/').split('/')[-1]
            text = re.sub(r'\W+', ' ', text).replace('html', '')
            if text is None or len(text.split()) <= 4:
                url_list.append(url)
            else:
                try:
                    detected_langs = detect_langs(text)
                except:
                    detected_langs = None

                langs = [x.lang for x in detected_langs]
                if ('en' not in langs) and len(langs) == 1:
                    print("pre_rejected: " + ' '.join(
                        [x.lang + '%.2f' % x.prob + ' ' for x in detected_langs]) + "   " + url)
                    print("\t" + text)
                    pre_rejected_list.append(url)
                else:
                    url_list.append(url)

        yield url_list, current_date
        with open(os.getcwd() + "\\" + directory_name + "\\" + query_name + "_pre_rejected.txt", "a+",
                  encoding="utf-8") as f:
            for pre_rejected in pre_rejected_list:
                f.write(pre_rejected)
                f.write('\n')

        current_date += timedelta(days=1)
        url_list.clear()

In [ ]:
start_date = date(2016, 12, 6)
end_date = date(2020, 4, 26)

directory_name = 'Articles'
try:
    os.mkdir(os.getcwd() + "\\" + directory_name)
except:
    print('Warning: Article directory already exists.')
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36'
config_browser = Config()
config_browser.browser_user_agent = user_agent

query = "Bitcoin"
query_name = query.split(' ')[-1]

In [ ]:
for urlList, current_date in Get_google_urls(query=query, quantity=1000, startdate=arrow.get(datetime(2016, 12, 6)),
                                             enddate=arrow.get(datetime(2020, 4, 26))):
    for http in urlList:
        article = Article(http, config=config_browser)
        try:
            article.download()
        except:
            print('Not parsed:' + http)

        if article.download_exception_msg != None:
            print("DOWNLOAD EXCEPTION: " + http)
            print(article.download_exception_msg)
        else:
            try:
                article.parse()
            except:
                print('Not parsed:' + http)
            title = 'None'
            title_stripped = 'None'

            # Tylko informacyjnie dla porównania. Data google wydaje się być lepszym rozwiązaniem.
            publish_date = article.extractor.get_publishing_date(url='', doc=article.clean_doc)
            if publish_date is not None:
                article_date = arrow.get(publish_date)
                if article_date.date() != current_date.date():
                    with open(os.getcwd() + "\\" + directory_name + "\\" + query_name + "_wrong_date.txt", "a+",
                              encoding="utf-8") as f:
                        print("wrong_date: " + http)
                        f.write(http)
                        f.write('\n')
                        f.write("GOOGLE: " + str(current_date.date()) + "  ARTICLE: " + str(article_date))
                        f.write('\n')
            # else:
            #     article_date = None

            article_date = current_date

            try:
                if len(article.text) > 500 and detect(article.text) == 'en' and article_date is not None:
                    if hasattr(article, 'title'):
                        title = article.title
                        title_stripped = ''.join(e for e in title[:50] if e.isalnum() or e in " $%")

                    directory = os.getcwd() + "\\" + directory_name + "\\" + article_date.strftime(
                        "%d.%m.%Y") + "\\" + query_name + "\\"
                    filename = directory + title_stripped + ".txt"
                    os.makedirs(os.path.dirname(filename), exist_ok=True)
                    with open(filename, "w", encoding="utf-8") as f:
                        f.write("DATE:  " + article_date.strftime("%d.%m.%Y, %H:%M:%S"))
                        f.write('\n')
                        f.write("TITLE: " + title)
                        f.write('\n')
                        f.write("URL:   " + http)
                        f.write('\n\n')
                        f.write(article.text)
                else:
                    with open(os.getcwd() + "\\" + directory_name + "\\" + query_name + "_rejected.txt", "a+",
                              encoding="utf-8") as f:
                        print("rejected: " + http)
                        f.write(http)
                        f.write('\n')
            except:
                print("exception: " +
                      article_date.strftime("%d.%m.%Y, %H:%M:%S") + "  " + http)
                with open(os.getcwd() + "\\" + directory_name + "\\" + query_name + "_exception.txt", "a+",
                          encoding="utf-8") as f:
                    f.write(http)
                    f.write('\n')
